# Problem Set 6 -- Solutions

## Problem 1. Solving linear systems

In [1]:
function lu_decomp(A)
    """ Return the LU decomposition without pivoting
    """
    N = size(A)[1]
    
    L = zeros(N, N)
    U = zeros(N, N)
    
    for i=1:N
        # diagonal of L
        L[i,i] = 1.0
        
        # columns of L
        for j=1:i-1
            L[i,j] = (A[i,j] - L[i,1:j-1]'*U[1:j-1,j])/U[j,j]
        end
        
        # rows of U
        for j=i:N
            U[i,j] = A[i,j] - L[i,1:i-1]'*U[1:i-1,j]
        end
    end
    
    return L, U
end

lu_decomp (generic function with 1 method)

In [2]:
function cholesky_decomp(A)
    """ Return the Cholesky decomposition
    of a symmetric positive definite matrix A
    """
    N = size(A)[1]
    
    L = zeros(N, N)
    
    for i=1:N
        # diagonal element
        L[i,i] = sqrt(A[i,i] - L[i,:]'*L[i,:])
        
        # off-diagonal elements
        for j=i+1:N
            L[j,i] = (A[i,j] - L[i,:]'*L[j,:])/L[i,i]
        end
    end
    
    return L
end

cholesky_decomp (generic function with 1 method)

In [3]:
function forward_subst(L, b)
    """ Solve Lx = b with L lower triangular
    using forward substitution
    """ 
    
    N = size(L)[1]
    x = zeros(N)
    
    for i=1:N
        x[i] = (b[i] - L[i,:]'*x)/L[i,i]
    end
    
    return x
end

forward_subst (generic function with 1 method)

In [4]:
function back_subst(U, b)
    """ Solve Ux = b with U upper triangular
    using backsubstitution
    """ 
    
    N = size(L)[1]
    x = zeros(N)
    
    for i=N:-1:1
        x[i] = (b[i] - U[i,:]'*x)/U[i,i]
    end
    
    return x
end

back_subst (generic function with 1 method)

### (a)

In [5]:
A = [1.0 3 2
2.4 -3.3 1.1
-1 0 2]

b = [1; 2; 3]

# This is a generic non-symetric matrix, so we use LU decomposition
L, U = lu_decomp(A)

([1.0 0.0 0.0; 2.4 1.0 0.0; -1.0 -0.285714 1.0], [1.0 3.0 2.0; 0.0 -10.5 -3.7; 0.0 0.0 2.94286])

In [6]:
# first solve L y = b using forward substitution
y = forward_subst(L, b)

3-element Array{Float64,1}:
  1.0               
 -0.3999999999999999
  3.8857142857142857

In [7]:
# then solve U x = y using backsubstitution
x = back_subst(U, y)

3-element Array{Float64,1}:
 -0.3592233009708736
 -0.4271844660194174
  1.320388349514563 

In [8]:
# compare to Julia's backslash operator
A \ b

3-element Array{Float64,1}:
 -0.3592233009708738 
 -0.42718446601941745
  1.320388349514563  

### (b)

In [197]:
A = [5 -3 2
-3 6 -1
2 -1 5]

b = [1; 2; 3]

3×3 Array{Int64,2}:
  5  -3   2
 -3   6  -1
  2  -1   5

In [198]:
# this is a symmetric matrix. If it's positive definite, then Cholesky works

L = cholesky_decomp(A)

3×3 Array{Float64,2}:
  2.23607   0.0      0.0    
 -1.34164   2.04939  0.0    
  0.894427  0.09759  2.04707

In [199]:
# Cholesky is successful, so we can solve
# Ly = b using forward substitution
y = forward_subst(L, b)

3-element Array{Float64,1}:
 0.4472135954999579
 1.2686700948330931
 1.2096294735180122

In [200]:
# and then Lᵀx = y using backsubstitution
x = back_subst(L', y)

3-element Array{Float64,1}:
 0.3181818181818181
 0.5909090909090908
 0.5909090909090909

In [201]:
# compare to backslash
A \ b

3-element Array{Float64,1}:
 0.31818181818181823
 0.590909090909091  
 0.5909090909090909 

### (c)

In [202]:
A = [2 -3 2
-3 3 -1
2 -1 2]

b = [1; 2; 3]

3-element Array{Int64,1}:
 1
 2
 3

In [203]:
# A symmetric again but Cholesky fails:
cholesky_decomp(A)

DomainError: DomainError with -1.4999999999999991:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [205]:
# So it's not positive definite. Therefore, we must resort to LU
L, U = lu_decomp(A)
y = forward_subst(L, b)
x = back_subst(U, y)

3-element Array{Float64,1}:
 -0.5
  1.0
  2.5

In [206]:
# compare to backslash
A \ b

3-element Array{Float64,1}:
 -0.5
  1.0
  2.5

## Problem 2. The QR decomposition.

### (a) Inverse of upper triangular matrix


We first show that $R^{-1}$ is upper triangular if $R$ is. This is easy to see using the
backsubstitution algorithm. The inverse of a matrix is computed by solving the linear systems

$$
R \mathbf{x}_i = \mathbf{e}_i,
$$
where $\mathbf{e}_i = (0, \dots, 0, \underbrace{1}_{i\text{th position}},0,\dots, 0)^\top$.
Then, $R^{-1} = [\mathbf{x}_1, \dots \mathbf{x}_n]$.

The backsubstitution algorithm then leads to
$$
(\mathbf{x}_i)_j = \frac{1}{R_{jj}} \left( (\mathbf{e}_i)_j - \sum_{k=j+1}^n R_{jk} (x_i)_k \right),
$$

since this is evaluated from the back from $j=N,N-1,N-2,\dots$, it will produce only zeros until
$j=i$. Therefore, the matrix $R^{-1}$ is again upper triangular if $R$ is upper triangular.

### (b) Product of upper triangular matrices

Let $X = R_1 R_2$ where $R_1, R_2$ are both upper triangular and square.
Then
$$
X_{ij} = \sum_{k=1}^N (R_1)_{ik} (R_2)_{kj}.
$$

The lower triangle if $X$ has $i < j$. But then $(R_1)_{ik}$ is zero for all $k$ from $1$ to $i-1$
and $(R_2)_{kj}$ is zero for all $k$ from $j+1$ to $N$.
Therefore all summands are zero for $i<j$ and the product is again upper triangular.

### (c) Uniqueness of the QR decomposition
Assume that that
$$
A = Q_1 R_1 = Q_2 R_2.
$$

It follows that
$$
Q_2^\top Q_1 = R_2 R_1^{-1}.
$$

We first check
$$
(Q_2^\top Q_1)^\top Q_2^\top Q_1 = Q_1^\top \underbrace{Q_2 Q_2^\top}_{=\mathbb{1}} Q_1 = Q_1^\top Q_1 = \mathbb{1},
$$
so the left hand side is again an orthogonal matrix.
By part (a), $R_1^{-1}$ is again upper triangular and by part (b), the RHS is a product of upper triangular
matrices and again upper triangular.

We now check that the only upper triangular matrix that is orthogonal is the identity.
Orthogonal matrices have orthogonal columns.

Compute the inner product of the first column $(r_{11}, 0, \dots, 0)^\top$ with all the others
to obtain
$$
1 = r_{11}^2, \quad 0 = r_{11} r_{21}, \quad 0 = r_{11} r_{31}, \quad\dots\quad, 0 = r_{11} r_{N1}.
$$
From this it follows that $r_{21} = r_{31} = \dots = r_{N1} = 0$.
Next, compute the inner product of the second column $(0, r_{22}, 0, \dots, 0)^\top$ with all the others,
$$
1 = r_{22}^2, \quad 0 = r_{22} r_{32}, \quad\dots\quad, 0 = r_{22} r_{N2}.
$$
From this it follows that $r_{32} = r_{42} = \dots = r_{N2} = 0$.
Continue this procedure to find that all off-diagonal elements $r_{ij}$,$i\neq j$ must be zero.

This leaves just the diagonal entries, which may all be $r_{ii} = \pm 1$.
But they are just $r_{ii} = (R_1)_{ii} (R_2)_{ii}$ because $R_1, R_2$ are upper triangular, and
since we had assumed that $R_1, R_2$ have positive diagonal entries, it follows that
$r_{ii} = 1$.

So,
$$
Q_1^\top Q_2 = \mathbb{1} \Rightarrow Q_2 = Q_1.
$$

### (d)
Since the $Q$ factor is unique we have
$$
Q_1^\top Q_2 = R_2 R_1^{-1} = \mathbb{1} \Rightarrow R_2 = R_1.
$$

Therefore, the QR decomposition with positive diagonal elements of $R$ is unique.

## Problem 3. Eigenvalues with the QR algorithm.

### (a)
We use the fact that
$$
\hat Q_k = Q_0 Q_1\dots Q_{k-1}
$$
converges to a set of orthonormal eigenvectors of $A$, if such a set exists.

Because each step of the QR algorithm produces one factor $Q_i$, we can simply keep
a running product that converges to a matrix of orthonormal eigenvectors.

In [207]:
function eig_qr(A; N=1000, ϵ=√eps())
    """ Compute eigenvalues of A using the QR algorithm
    """
    F = qr(A)
    Q, R = F.Q, F.R
    
    # keep running product of all the Q's
    Q̂ = Q
    
    for i=1:N
        # new matrix
        A_new = R*Q
        
        # termination criterion
        if norm(A_new - A) < ϵ*norm(A_new)
            break
        end
        
        A = A_new
        
        # QR decomposition
        F = qr(A)
        Q, R = F.Q, F.R
        
        # update running product
        Q̂ = Q̂*Q
    end
    
    # return eigenvalues and eigenvectors
    return diag(A), Q̂
end

eig_qr (generic function with 1 method)

### (b)

In [209]:
# test algorithm
A = [2 -3 2
-3 1 4
2 4 -1]

u, v = eig_qr(A)

([-5.6758, 4.91639, 2.75942], [0.412659 -0.560236 0.718226; 0.597434 0.761669 0.250865; -0.687594 0.325571 0.649013])

In [211]:
# check that they are eigenvectors
A*v./v

3×3 Array{Float64,2}:
 -5.6758  4.91639  2.75942
 -5.6758  4.91639  2.75942
 -5.6758  4.91639  2.75942

In [212]:
# check that they are orthonormal
v'*v

3×3 Array{Float64,2}:
 1.0           3.02536e-15   1.66533e-15
 3.02536e-15   1.0          -4.16334e-16
 1.66533e-15  -4.16334e-16   1.0        

### (c)

In [272]:
# test algorithm
B = [2 -3 1
-3 1 4
2 4 -1]

u, v = eig_qr(B)

([-5.38038, 5.07159, 2.30878], [0.344519 -0.636 0.690514; 0.609579 0.710942 0.350677; -0.713947 0.300108 0.632626])

In [273]:
B*v./v

3×3 Array{Float64,2}:
 -5.38038  4.88163  1.39262
 -5.38038  5.37227  2.30878
 -5.38038  4.23734  3.40029

In [274]:
# The columns are not eigenvectors anymore!

In [275]:
v'*v

3×3 Array{Float64,2}:
  1.0           6.38378e-16  -1.83187e-15
  6.38378e-16   1.0          -2.72005e-15
 -1.83187e-15  -2.72005e-15   1.0        

In [217]:
# The matrix is still orthogonal

This can be explained by noting that B is not symmetric and therefore
not guaranteed to possess orthogonal eigenvectors.
While Orthogonal Iteration only converges to orthogonal eigenvectors if orthogonal eigenvectors exist,
the QR algorithm will always converge to the correct eigenvalues, if it converges.

We can compute normalized eigenvectors with Julia and see that they are not
orthogonal.

In [225]:
v = eigvecs(B)

3×3 Array{Float64,2}:
  0.344519  0.772971  -0.62409 
  0.609579  0.127599   0.730983
 -0.713947  0.621478   0.275999

In [227]:
# these are the correct eigenvectors
B*v./v

3×3 Array{Float64,2}:
 -5.38038  2.30878  5.07159
 -5.38038  2.30878  5.07159
 -5.38038  2.30878  5.07159

In [229]:
# but they are not orthogonal
v'*v

3×3 Array{Float64,2}:
  1.0        -0.099617   0.0335324
 -0.099617    1.0       -0.217604 
  0.0335324  -0.217604   1.0      

## Problem 4. Eigenvectors using Inverse Iteration
### (a)
We have
$$
A \mathbf{v} = \lambda \,\mathbf{v}.
$$

Then,
\begin{align}
(A - \mu \mathbb{1})^{-1}\mathbf{v} &= \sigma \mathbf{v} \\
\Rightarrow \frac{1}{\sigma} \mathbf{v} &= (A - \mu \mathbb{1}) \mathbf{v} = A \mathbf{v} - \mu\mathbf{v} \\
\Rightarrow A\mathbf{v} &= \left(\frac{1}{\sigma} + \mu\right) \mathbf{v}.
\end{align}

So if $\lambda$ is an eigenvalue of A, $\sigma = \frac{1}{\lambda  - \mu}$ is an eigenvalue of $(A - \mu\mathbb{1})^{-1}$ with the same eigenvector.

### (b)
Inverse iteration is defined by

$$
\mathbf{b}_{k+1} = \frac{(A - \mu \mathbb{1})^{-1} \mathbf{b}_k}{\|(A - \mu \mathbb{1})^{-1} \mathbf{b}_k\|}.
$$

Define $B = (A - \mu \mathbb{1})^{-1}$, then inverse iteration is just power iteration, and will
converge to an eigenvector for the dominant eigenvalue of $B$.

By part (a), the largest (dominant) eigenvalue of $B$ is the one where $\lambda -\mu$ is smallest, which
is the one closest to $\mu$, and the iteration
converges to an eigenvector for that eigenvalue.

### (c)

In [276]:
function inverse_iteration(A, μ; N=10000, ϵ=√eps())
    """ Perform inverse iteration to find the eigenvector 
    of A for the eigenvalue closest to μ.
    """
    n = size(A)[1]
    
    # random initial conditions
    v = randn(n)
    
    for i=1:N
        # inverse iteration
        v_new = (A - μ*I) \ v
        v_new = v_new/norm(v_new)
        
        # termination criterion 
        if norm(v_new - v) < ϵ
            return v_new
        end
        
        v = v_new
    end
    
    return v
end

function eigvecs_inverse_iteration(A, λs)
    """ Find eigenvectors for all eigenvalues λs of 
    the general square matrix A
    """
    N = size(A)[1]
    
    # sort eigenvalues from smallest to largest
    sort!(λs)
    
    v = zeros(N, N)
    
    # construct μs such that they all are closest to one of the
    # λs.
    μs = copy(λs)
    
    # smallest μ is 98% from smallest eigenvalue
    μs[1] *= 0.98
    
    # all others are 98% of the difference, so they are very close but below
    # this guarantees the they are always closest to just one eigenvalue.
    μs[2:end] = μs[1:end-1] + 0.98*diff(λs)
    
    for (i, μ) in enumerate(μs)
        v[:,i] = inverse_iteration(A, μ)
    end
    
    return v
end

eigvecs_inverse_iteration (generic function with 1 method)

In [277]:
# test iteration

B = [2 -3 1
-3 1 4
2 4 -1]

# eigenvalues from QR algorithm
u, _ = eig_qr(A)

([-5.38038, 5.07159, 2.30878], [0.344519 -0.636 0.690514; 0.609579 0.710942 0.350677; -0.713947 0.300108 0.632626])

In [278]:
v = eigvecs_inverse_iteration(A, u)

3×3 Array{Float64,2}:
 -0.344519  -0.772971  -0.62409 
 -0.609579  -0.127599   0.730983
  0.713947  -0.621478   0.275999

In [279]:
# check that they are eigenvectors
A*v./v

3×3 Array{Float64,2}:
 -5.38038  2.30878  5.07159
 -5.38038  2.30878  5.07159
 -5.38038  2.30878  5.07159

In [280]:
# they are not orthogonal!
v'*v

3×3 Array{Float64,2}:
  1.0        -0.099617  -0.0335324
 -0.099617    1.0        0.217604 
 -0.0335324   0.217604   1.0      